In [ ]:
import torch


stuff = torch.load("checkpoint_100.pt",
                   map_location=torch.device('cpu') 
                   )

In [ ]:
!cd src

In [ ]:
print(stuff.keys())
from ignite.handlers import Checkpoint

from train import Config, all_parts
from model import BSRNN

stuff["cfg"] = Config(
  {
        "--sample_rate": 44100,
         "--n_fft": 2048,
         "--generic_bands": True,
         "--chunk_size_in_seconds": 1,
         "--feature_dim": 128//4,
         "--data_loader_workers": 1,
         "--num_blstm_layers": 24//2,
         "--mlp_dim": 128//4,
         "--batch_size": 95,
         "--clip_grad_norm": 5,
         "--max_epochs": 100,
         "--portion": 1.0,
         "--lr": 0.001,
         "--musdbhq_location": "./musdb18-hq",
         "--checkpoint_fp": "./point.pt",
         "--name": "baseline",
         "--reduce_size": True,
         "parts": ["drums"],
  }

)

model = stuff["cfg"].create_model(len(["drums"]))

model_keys = list(stuff["model"].keys())

# Convert to the new version
for k in model_keys:
    if 'mask.layers' in k:
        stuff["model"][k.replace("mask.layers", "masks.0.layers")] = stuff["model"][k]
        del stuff["model"][k]

model.load_state_dict(stuff["model"])



In [ ]:
import torchaudio
import utils

utils.GLOBAL_CONFIG = utils.GlobalConfig({"--seed": 42, "--device": "cpu"})

waveform = utils.load_audio("/home/sahand/Pomplamoose - Nuclear Kittens ça devient chaud.mp3", cfg=stuff["cfg"])

normal_waveform, gain_factor, peak_gain_factor = utils.normalize_waveform(
        waveform, cfg=stuff["cfg"]
    )
splits, padding_length = utils.split(normal_waveform, cfg=stuff["cfg"])
to_spectrogram = utils.to_spectrogram(stuff["cfg"])
from_spectrogram = utils.from_spectrogram(stuff["cfg"])

split_stft = to_spectrogram(splits[0])
print(splits.shape)
y = model(split_stft.unsqueeze(0))


for i, part in enumerate(all_parts):
    # torch.Size([4, 1, 1025, 87])
    print(f"part {part}:")
    part_stft = y[i][0]
    wav = from_spectrogram(part_stft)
    print(wav)
    utils.show_idp_audio(wav.detach().numpy(), stuff["cfg"])



In [ ]:
torch.save(stuff, "../models/baseline-drums-to-vocals.pt")